# Car Price Prediction:

Download dataset from this link:

https://www.kaggle.com/hellbuoy/car-price-prediction

# Problem Statement::

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

Which variables are significant in predicting the price of a car
How well those variables describe the price of a car
Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

# task::
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

# WORKFLOW ::

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

5.Compilation Step (Note : Its a Regression problem , select loss , metrics according to it)
6.Train the Model with Epochs (100) and validate it

7.If the model gets overfit tune your model by changing the units , No. of layers , activation function , epochs , add dropout layer or add Regularizer according to the need .

8.Evaluation Step

9.Prediction

# 1. Load Data
Loading and immediately shuffeling Data frame.

In [ ]:
# Prediction Model developed by:
# Khurram Nazir
# khurram.deutsch@yahoo.com
# Munich,Germany.
#Comments and suggestions are welcomed.
#
import pandas as pd
import numpy as np
import seaborn as sns
#
import io
#from google.colab import files
#uploaded_file = files.upload()

df = pd.read_csv('../input/car-price-prediction/CarPrice_Assignment.csv')
#df = pd.read_csv("C:/Users/khurr/Documents/GitHub/AI-Engineering/PIAIC/Quarter-2/DeepLearning/Car_Price_Prediction_Assignment/CarPrice_Assignment.csv",sep=',')
#df = pd.DataFrame(pd.read_csv(io.BytesIO(uploaded_file['CarPrice_Assignment.csv']),sep=','))

In [ ]:

df = df.sample(frac=1) #Shuffeling DF.
df

In [ ]:
def df_lookslike(v_df):
    #---------------------------------------How dataframe looks like?
    v_df.info()
    print(v_df.head(5))
    total_cells=np.product(v_df.shape)
    num_col = [i for i in v_df.columns if (v_df[i].dtype=='int64' or v_df[i].dtype=='float64')]
    print(v_df[num_col].describe().loc[['min','max', 'mean','50%'],:]) #How big is Messy data?
    missing_Values=v_df.isnull().sum()
    print(missing_Values)
    total_missing=missing_Values.sum()

    #Percent of Missing data
    print("Percent of data is missing:",((total_missing/total_cells) * 100))

# 2. Check Missing Values
( If Exist ; Fill each record with mean of its feature )

I found there is NO missing/NULL data.

In [ ]:
df_lookslike(df)# How DF looks like?

# Preparing Tensor / Vectorization

In [ ]:
#Following libarary is performing OneHotEncoding. BEST thing: Its also modifying data types of target columns.
from sklearn.preprocessing import LabelEncoder
label_enc=LabelEncoder()


#Encoding scheme HotEncoding by using SKlearn. Let's seee further Heatmap to drop unneeded columns.
df['CarName']=label_enc.fit_transform(df['CarName'])
df['fueltype']=label_enc.fit_transform(df['fueltype'])
df['aspiration']=label_enc.fit_transform(df['aspiration']) 
df['doornumber']=label_enc.fit_transform(df['doornumber'])
df['carbody']=label_enc.fit_transform(df['carbody'])
df['drivewheel']=label_enc.fit_transform(df['drivewheel'])
df['enginelocation']=label_enc.fit_transform(df['enginelocation'])
df['enginetype']=label_enc.fit_transform(df['enginetype'])
df['cylindernumber']=label_enc.fit_transform(df['cylindernumber'])
df['fuelsystem']=label_enc.fit_transform(df['fuelsystem'])

df_lookslike(df)# Post OnehotEncoding let see, how DF looks like?


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corrmat = df.corr(method='pearson',min_periods=5) #standard correlation coefficient,
top_corr_features = corrmat.index
plt.figure(figsize=(30,30))
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
#I'm going to drop few features/Indirect variable which has -ve correlation/association with Price.
df.drop(columns=['highwaympg','citympg','car_ID','CarName','symboling','enginelocation'], inplace=True)
df.head(10)

# 3. Split Data.

50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).


In [ ]:

from sklearn.model_selection import train_test_split

Y=(df.loc[:,df.columns=='price']) #Lets take Dependent Variable/Target in a serpate df i.e X.
X=(df.loc[:,df.columns!='price']) #Lets take Independent Variables in a serpate df i.e Y.


x_train_50,X_remaining,y_train_50,Y_remaining=train_test_split(X,Y,test_size=0.5,random_state=0)
x_test_30,x_valid_20,y_test_30,y_valid_20=train_test_split(X_remaining,Y_remaining,test_size=0.7,random_state=0)

print("Training Data-X [50%]:\t", x_train_50.shape)
print("Test Data-X [30%]:\t", x_test_30.shape)
print("Validate Data-X [20%]:\t", x_valid_20.shape)

# 4. Data Normalization


In [ ]:

#Training data normalization manually.
#Its not encouraged to Normalize all DF. Rather, to only qualified columns. Need more discussion/comments.

mean = x_train_50.iloc[: , 0:14].mean(axis=0)
x_train_50.iloc[: , 0:14] -= mean
std = x_train_50.iloc[:, 0:14].std(axis=0)
x_train_50.iloc[: , 0:14] /= std
x_train_50



In [ ]:
#Validation data normalization
val_mean = x_valid_20.iloc[:, 0:14].mean(axis=0)
val_std = x_valid_20.iloc[:, 0:14].std(axis=0)
x_valid_20.iloc[:, 0:14] -= val_mean
x_valid_20.iloc[:, 0:14] /= val_std
x_valid_20

In [ ]:
#Test data normalization
test_mean = x_test_30.iloc[:, 0:14].mean(axis=0)
test_std =  x_test_30.iloc[:, 0:14].std(axis=0)
x_test_30.iloc[:, 0:14] -= test_mean
x_test_30.iloc[:, 0:14] /= test_std
x_test_30

# 5. Model :

Input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout

network = Sequential()



network.add(layers.Dense(16, activation='relu', kernel_regularizer =regularizers.l2(0.02),   input_shape=(x_train_50.shape[1],)))
network.add(layers.Dense(14, activation='relu', kernel_regularizer =regularizers.l2(0.02)))
network.add(layers.Dense(8, activation='relu', kernel_regularizer =regularizers.l2(0.02)))
network.add(layers.Dense(6, activation='relu', kernel_regularizer =regularizers.l2(0.002)))



#I'm passing 01-Neuron but do not specify Activation function as its Regression.
network.add(layers.Dense(1))


# 6.Compilation
Steps(Note : Its a Regression problem , select loss , metrics according to it)


In [ ]:
from tensorflow import keras
#Preparing parameters for Optimizer.
opt = keras.optimizers.RMSprop() #I want lowest learning rate as higher accuracy required.
network.compile(optimizer=opt, loss='mean_absolute_error', metrics='mae')

# 7.Train Model
with Epochs (100) and validate it



In [ ]:
history = network.fit(x_train_50,y_train_50, batch_size=16, verbose=0, epochs=300, validation_data=(x_valid_20, y_valid_20))

In [ ]:
epochs = range(1, 301)
train_mae = history.history['mae']
val_mae = history.history['val_mae']

In [ ]:
plt.plot(epochs, train_mae, 'r', label='Training MAE')
plt.plot(epochs, val_mae, 'g', label='Validation MAE')
plt.title('Training vs Validation Loss ')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.plot(epochs, train_loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Training vs Validation loss ')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
print('Predicted Price:\n',network.predict(x_test_30))

# 8.Evaluation Step

In [ ]:
test_mse_score, test_mae_score  =network.evaluate(x_test_30, y_test_30)

In [ ]:
#Please comment specially w.r.t Normalization.
#khurram.deutsch@yahoo.com